## Constrained Least Squares

In [1]:
# Import necessary libraries
import numpy as np
# Set seed to reproduce results
np.random.seed(40)

In [2]:
# Random matrices for debugging
A_ = np.array([[0.45035059, 0.30391231, 0.52639952, 0.62381221],
       [0.77677546, 0.68624165, 0.98093886, 0.60081609],
       [0.81396852, 0.70864515, 0.02753468, 0.90426722],
       [0.44990485, 0.11892465, 0.83530018, 0.20224823]])
x_ = np.array([0.40768703, 0.05536604, 0.78853488, 0.28730518])

### Back Substitution
The process of solving a linear system of equations that has been transformed into row-echelon form or reduced row-echelon form.

The last row is solved first, followed by second last row and so on.

#### General formula
x_i = (b_i - A[i, i + 1] * x[i + 1] - ... - A[i, n] * x[n]) / A[i, i]

In [3]:
def backsubstitution(A: np.ndarray, b: np.ndarray) -> np.ndarray :
    """Solves a system of linear equations using back substitution. Assumption is that
        A is upper triangular

    Parameters
    ----------
    A : ndarray(dtype=float, ndim=2)
        Upper triangular matrix with non zero diagonal elements.
    b : ndarray(dtype=float, ndim=1)
    
    Returns
    ----------
    x : ndarray(dtype=float, ndim=1)
        Solution to the system of equations
   """
    
    n, m = A.shape
    x = np.zeros(n)
    
    for i in np.arange(n - 1, -1, -1) :
        
        x[i] = b[i] - np.sum(np.dot(A[i, i+1 : n ], x[i+1 : n]))
        x[i] = x[i] / A[i, i]
        
    return x

In [4]:
# Compare the value returned by the function with original x
# The difference should be small
A = np.triu(A_)
x = x_

b = A @ x
xhat = backsubstitution(A, b)
print(np.sum(x - xhat))


-6.938893903907228e-17


Expected Output : -6.938893903907228e-17

### Gram - Schmidt Orthonormalization
The Gram–Schmidt process is a method for orthonormalizing a set of vectors.

General Formula :

q_i = (a_i - (q_0'a_0)q_0 .... - (q_(i-1)'a_(i-1))q_(i-1))

q_i = q_i / |q_i|

In [5]:
def gram_schmidt(A: np.ndarray) -> np.ndarray :
    """Makes the matrix orthogonal using the Gram-Schmidt ortogonalisation process.

    Parameters
    ----------
    A : ndarray(dtype=float, ndim=2)
        Matrix with non linear rows.
    
    Returns
    ----------
    Q : ndarray(dtype=float, ndim=2)
        Ortogonal form of A.
   """
    
    Q = np.zeros_like(A, dtype = 'float')
    n, m = A.shape
    
    for i in range(n) :
        Q[i] = A[i] - sum((Q[k].T @ A[i]).reshape(-1, 1) * Q[k] for k in range(i))
        #Q[i] = A[i] - np.sum((Q[: i] @ A[i].T).reshape(-1, 1) * Q[: i], 1)

        if np.sqrt(np.sum(Q[i] ** 2)) <= 1e-10 :
            raise Exception("Rows are linearly dependent")
        else :
            Q[i] = Q[i] / np.sqrt(np.sum(Q[i] ** 2))
    
    return Q 

In [6]:
a = np.array([ [-1, 1, -1, 1], [-1, 3, -1, 3], [1, 3, 5, 7] ])
q = gram_schmidt(a)
print(q)
#Test orthonormality
print('Norm of q[0] :', (sum(q[0]**2))**0.5)
print('Inner product of q[0] and q[2] :', q[0] @ q[2])


[[-0.5  0.5 -0.5  0.5]
 [ 0.5  0.5  0.5  0.5]
 [-0.5 -0.5  0.5  0.5]]
Norm of q[0] : 1.0
Inner product of q[0] and q[2] : 0.0


Expected Output :
[[-0.5  0.5 -0.5  0.5]
 [ 0.5  0.5  0.5  0.5]
 [-0.5 -0.5  0.5  0.5]]
 
Norm of q[0] : 1.0

Inner product of q[0] and q[2] : 0.0


### QR Factorization

In [7]:
def QR_factorization(A: np.ndarray) -> (np.ndarray, np.ndarray) :
    """Performs QR factorization of matrix A.

    Parameters
    ----------
    A : ndarray(dtype=float, ndim=2)
        Matrix with non linear columns.
    
    Returns
    ----------
    Q : ndarray(dtype=float, ndim=2)
    R : ndarray(dtype=float, ndim=2)
   """
    Q = gram_schmidt(A.T).T
    # Code to calculate R matrix in QR factorization
    R = Q.T @ A
    
    return Q, R

In [8]:
A = A_
Q1, R1 = QR_factorization(A)
Q2, R2 = np.linalg.qr(A)

# print("Q1 and Q2 : ")
# print(Q1, Q2)
# print("R1 and R2 : ")
# print(R1, R2)


Q1 and Q2 : 

[[ 3.48371311e-01 -1.78310119e-01  8.11429912e-02  9.16656286e-01]
 [ 6.00879163e-01  3.14060516e-01  6.98444699e-01 -2.29096543e-01]
 [ 6.29650071e-01  2.88993757e-01 -7.11048771e-01 -1.20137599e-01]
 [ 3.48026507e-01 -8.86596922e-01 -6.80545069e-04 -3.04668648e-01]] 
 
 [[-3.48371311e-01  1.78310119e-01  8.11429912e-02 -9.16656286e-01]
 [-6.00879163e-01 -3.14060516e-01  6.98444699e-01  2.29096543e-01]
 [-6.29650071e-01 -2.88993757e-01 -7.11048771e-01  1.20137599e-01]
 [-3.48026507e-01  8.86596922e-01 -6.80545069e-04  3.04668648e-01]]
 
R1 and R2 : 

[[ 1.29273156e+00  1.00581004e+00  1.08085203e+00  1.21809581e+00]
 [-1.80411242e-15  2.60686562e-01 -5.18405414e-01  1.59475505e-01]
 [-1.88737914e-15 -8.32667268e-16  7.07698218e-01 -1.72860933e-01]
 [-2.22044605e-15  3.05311332e-16 -2.05391260e-15  2.63921306e-01]] 
 
 [[-1.29273156 -1.00581004 -1.08085203 -1.21809581]
 [ 0.         -0.26068656  0.51840541 -0.1594755 ]
 [ 0.          0.          0.70769822 -0.17286093]
 [ 0.          0.          0.         -0.26392131]]


### Solving Linear Equations

In [9]:
def solve_linear_equations(A: np.ndarray, b: np.ndarray) -> np.ndarray :
    """Solves the system of linear equations Ax = b.

    Parameters
    ----------
    A : ndarray(dtype=float, ndim=2)
    b : ndarray(dtype=float, ndim=1)
    
    Returns
    ----------
    x : ndarray(dtype=bool, ndim=1)
    solution to the system of linear equations Ax = b
   """    
    # QR factorization of A
    Q, R = QR_factorization(A)
    b = Q.T @ b
    # back substitution to calculate x
    x = backsubstitution(R, b)
    
    return x

In [10]:
A = A_
x = x_

b = A @ x
xhat = solve_linear_equations(A, b)
print(np.sum(x - xhat))


1.824929096727601e-15


Expected Output : 1.908195823574488e-15

### Solving Constrained Least Squares

#### Solving via KKT

In [15]:
def solve_cls_kkt(A: np.ndarray, b: np.ndarray, C: np.ndarray, d: np.ndarray) :
    """Solved the Constrained Least Squares problem using KKT matrix.

    Parameters
    ----------
    A : ndarray(dtype=float, ndim=2)
    b : ndarray(dtype=float, ndim=1)
    C : ndarray(dtype=float, ndim=2)
    d : ndarray(dtype=float, ndim=1)
    
    Returns
    ----------
    x : ndarray(dtype=float, ndim=1)
    Solution for cls satisfying Cx = d while minimizing |Ax - b|^2
   """    
    
    m, n = A.shape
    p, n = C.shape
    
    # General form
    # 2 * (A'A) x + 2 * A'b + C'z = 0
    
    # Gram matrix
    G = A.T @ A
    # KKT matrix

    # TODO : Create the KKT matrix as described in the algorithm
    # Hint : use np.vstack, np.hstack or np.concatenate
    z = np.zeros([p,p])
    row1 = np.hstack((2*G,C.T))
    row2 = np.hstack((C,z))
    KKT = np.vstack((row1,row2))

    
    x = solve_linear_equations(KKT, np.hstack([2*A.T @ b,d]))
    return x[: n]

In [12]:
# The residual Cx - d should be low

A = np.array([[-0.19900912, -1.27498361,  0.29349415,  0.10895031,  0.03172679],
       [ 1.27263986,  1.0714479 ,  0.41581801,  1.55067923, -0.31137892],
       [-1.37923991,  1.37140879,  0.02771165, -0.32039958, -0.84617041],
       [-0.43342892, -1.3370345 ,  0.20917217, -1.4243213 , -0.55347685],
       [ 0.07479864, -0.50561983,  1.05240778,  0.97140041,  0.07683154],
       [-0.43500078,  0.5529944 ,  0.26671631,  0.00898941,  0.64110275],
       [-0.17770716,  0.69627761, -1.1887251 , -0.33169686,  0.03007614],
       [-1.10791517, -0.5499249 , -2.03290956,  1.4079178 ,  0.63310826],
       [ 2.21274689, -0.52660228,  0.54288168, -0.0844797 ,  1.29201502],
       [-0.17671057,  1.68778715, -1.04661354,  0.64212021, -0.17296174]])
b = np.array([-1.11206497, -0.02070278, -1.81352177,  0.20352189,  0.53187404,
       -1.01702346, -1.84805054,  0.17254907,  0.78620153, -0.07147003])
C = np.array([[ 0.83634877, -0.19822889, -0.09227256,  0.87072588, -0.83919248],
       [ 0.82325537,  0.70175236,  0.59836051, -0.50799497,  0.13613812]])
d = np.array([-1.49211459,  0.61166554])

x1 = solve_cls_kkt(A, b, C, d)
print(C @ x1 - d)
print(x1)


[3.33066907e-15 5.10702591e-15]
[-0.0982787   0.12220912  0.48788039 -0.26556527  1.3220345 ]


Expected Output : 
    
[8.65973959e-15 6.66133815e-16]

[-0.0982787   0.12220912  0.48788039 -0.26556527  1.3220345 ]

#### Solving via QR

In [13]:
# Assign the appropriate values for the variables used below

FUNCTION1 = backsubstitution
FUNCTION2 = QR_factorization
FUNCTION3 = np.vstack


def solve_cls_QR(A: np.ndarray, b: np.ndarray, C: np.ndarray, d: np.ndarray) :
    """Solved the Constrained Least Squares problem using KKT matrix.

    Parameters
    ----------
    A : ndarray(dtype=float, ndim=2)
    b : ndarray(dtype=float, ndim=1)
    C : ndarray(dtype=float, ndim=2)
    d : ndarray(dtype=float, ndim=1)
    
    Returns
    ----------
    x : ndarray(dtype=float, ndim=1)
    Solution for cls satisfying Cx = d while minimizing |Ax - b|^2
   """    
    m, n = A.shape
    p, n = C.shape

    # step 1
    Q, R = QR_factorization(FUNCTION3([A,C]))  
    Q1 = Q[: m, :]
    Q2 = Q[m : m + p + 1, :]
    
    Qtil, Rtil = FUNCTION2(Q2.T)
    
    
    # step 2 + 3
    w = FUNCTION1(Rtil, (2 * Qtil.T @ (Q1.T @ b) - 2 * solve_linear_equations(Rtil.T, d)))
    # step 4
    x = FUNCTION1(R, (Q1.T @ b - (Q2.T @ w) / 2))
    
    return x

In [14]:
# The residual Cx - d should be low
# Comparing the results of two methods,
# the norm of x1 - x2 should be low
x2 = solve_cls_QR(A, b, C, d)
print(C @ x2 - d)
print(x2)
print(np.linalg.norm(x1 - x2))


[ 0.00000000e+00 -2.22044605e-16]
[-0.0982787   0.12220912  0.48788039 -0.26556527  1.3220345 ]
5.377781551615544e-15


Expected Output :

[0.00000000e+00 1.11022302e-16]

[-0.0982787   0.12220912  0.48788039 -0.26556527  1.3220345 ]

7.150711129076838e-15